*Copyright (c) Microsoft Corporation. All rights reserved.*  
*Licensed under the MIT License.*
# Named Entity Recognition Using BERT on Chinese
## Summary
This notebook demonstrates how to fine tune [pretrained BERT model](https://github.com/huggingface/pytorch-pretrained-BERT) for named entity recognition (NER) task on Chinese text. Utility functions and classes in the NLP Best Practices repo are used to facilitate data preprocessing, model training, and model evaluation. 

[BERT (Bidirectional Transformers for Language Understanding)](https://arxiv.org/pdf/1810.04805.pdf) is a powerful pre-trained lanaguage model that can be used for multiple NLP tasks, including text classification, question answering, named entity recognition, etc. It's able to achieve state of the art performance with only a few epochs of fine tuning on task specific datasets.  
The figure below illustrates how BERT can be fine tuned for NER tasks. The input data is a list of tokens representing a sentence. In the training data, each token has an entity label. After fine tuning, the model predicts an entity label for each token in a given testing sentence. 

<img src="https://nlpbp.blob.core.windows.net/images/bert_architecture.png">

## Required packages
* pytorch
* pytorch-pretrained-bert
* pandas
* seqeval

In [1]:
import sys
import os
import random
from seqeval.metrics import f1_score, classification_report

import torch
from pytorch_pretrained_bert.tokenization import BertTokenizer

nlp_path = os.path.abspath('../../')
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

from utils_nlp.bert.token_classification import BERTTokenClassifier, postprocess_token_labels, create_label_map
from utils_nlp.bert.common import Language, Tokenizer
from utils_nlp.dataset.msra_ner import load_pandas_df, get_unique_labels

## Configurations

In [2]:
# path configurations
DATA_DIR = "./data"
TRAIN_DATA_FILE = "../MSRA/msra-bakeoff3-training-utf8.2col"
Test_DATA_FILE = "../MSRA/bakeoff3_goldstandard.txt"
CACHE_DIR="./temp"

# set random seeds
RANDOM_SEED = 100
torch.manual_seed(RANDOM_SEED)

# model configurations
LANGUAGE = Language.CHINESE
DO_LOWER_CASE = True
MAX_SEQ_LENGTH = 200

# training configurations
BATCH_SIZE = 16
NUM_TRAIN_EPOCHS = 1

# optimizer configuration
LEARNING_RATE = 3e-5
WARMUP_PROPORTION = 0.1

TEXT_COL = "sentence"
LABEL_COL = "labels"

CACHE_DIR = "../../../"

## Preprocess Data

### Get training and testing data
The dataset used in this notebook is the [wikigold dataset](https://www.aclweb.org/anthology/W09-3302). The wikigold dataset consists of 145 mannually labelled Wikipedia articles, including 1841 sentences and 40k tokens in total. The dataset can be directly downloaded from [here](https://github.com/juand-r/entity-recognition-datasets/tree/master/data/wikigold). The `download` function downloads the data file to a user-specified directory.  

The helper function `get_train_test_data` splits the dataset into training and testing sets according to `test_percentage`. Because this is a relatively small dataset, we set `test_percentage` to 0.5 in order to have enough data for model evaluation. Running this notebook multiple times with different random seeds produces similar results.   

The helper function `get_unique_labels` returns the unique entity labels in the dataset. There are 5 unique labels in the   original dataset: 'O' (non-entity), 'I-LOC' (location), 'I-MISC' (miscellaneous), 'I-PER' (person), and 'I-ORG' (organization). An 'X' label is added for the trailing word pieces generated by BERT, because BERT uses WordPiece tokenizer.  

The maximum number of words in a sentence is 144, so we set MAX_SEQ_LENGTH to 200 above, because the number of tokens will grow after WordPiece tokenization.

In [3]:
train_df = load_pandas_df(local_cache_path=CACHE_DIR, file_split="train")
test_df = load_pandas_df(local_cache_path=CACHE_DIR, file_split="test")
label_list = get_unique_labels()
print("Number of sentences in training data: {}".format(train_df.shape[0]))
print("Number of sentences in testing data: {}".format(test_df.shape[0]))
print("Unique labels: {}".format(label_list))

Maximum sequence length in train data is: 746
Maximum sequence length in test data is: 439
Number of sentences in training data: 45000
Number of sentences in testing data: 3940
Unique labels: ['O', 'B-LOC', 'B-ORG', 'B-PER', 'I-LOC', 'I-ORG', 'I-PER']


In [4]:
train_df.head()

,sentence,labels
0,当 希 望 工 程 救 助 的 百 万 儿 童 成 长 起 来 ， 科 教 兴 国 蔚 然 ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,藏 书 本 来 就 是 所 有 传 统 收 藏 门 类 中 的 第 一 大 户 ， 只 是 ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,因 有 关 日 寇 在 京 掠 夺 文 物 详 情 ， 藏 界 较 为 重 视 ， 也 是 ...,"[O, O, O, B-LOC, O, O, B-LOC, O, O, O, O, O, O..."
3,我 们 藏 有 一 册 １ ９ ４ ５ 年 ６ 月 油 印 的 《 北 京 文 物 保 存 ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,以 家 乡 的 历 史 文 献 、 特 定 历 史 时 期 书 刊 、 某 一 名 家 或 ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


### Tokenization and Preprocessing
The `preprocess_ner_tokens` method of the `Tokenizer` class converts raw string data to numerical features, involving the following steps:
1. WordPiece tokenization.
2. Convert tokens and labels to numerical values, i.e. token ids and label ids.
3. Sequence padding or truncation according to the `max_seq_length` configuration.

**Create a dictionary that maps labels to numerical values**

In [5]:
label_map = create_label_map(label_list)

**Tokenize input text**

In [6]:
tokenizer = Tokenizer(language=LANGUAGE, 
                      to_lower=DO_LOWER_CASE, 
                      cache_dir=CACHE_DIR)

**Create numerical features**  

In [9]:
train_token_ids, train_input_mask, train_trailing_token_mask, train_label_ids = \
    tokenizer.tokenize_preprocess_ner_text(text=train_df[TEXT_COL],
                                           label_map=label_map,
                                           max_len=MAX_SEQ_LENGTH,
                                           labels=train_df[LABEL_COL])
test_token_ids, test_input_mask, test_trailing_token_mask, test_label_ids = \
    tokenizer.tokenize_preprocess_ner_text(text=test_df[TEXT_COL],
                                           label_map=label_map,
                                           max_len=MAX_SEQ_LENGTH,
                                           labels=test_df[LABEL_COL])

`Tokenizer.preprocess_ner_tokens` outputs three or four lists of numerical features lists, each sublist contains features of an input sentence: 
1. token ids: list of numerical values each corresponds to a token.
2. attention mask: list of 1s and 0s, 1 for input tokens and 0 for padded tokens, so that padded tokens are not attended to. 
3. trailing word piece mask: boolean list, `True` for the first word piece of each original word, `False` for the trailing word pieces, e.g. ##ing. This mask is useful for removing predictions on trailing word pieces, so that each original word in the input text has a unique predicted label. 
4. label ids: list of numerical values each corresponds to an entity label, if `labels` is provided.

In [10]:
print("Sample token ids:\n{}\n".format(train_token_ids[0]))
print("Sample attention mask:\n{}\n".format(train_input_mask[0]))
print("Sample trailing token mask:\n{}\n".format(train_trailing_token_mask[0]))
print("Sample label ids:\n{}\n".format(train_label_ids[0]))

Sample token ids:
[2496, 2361, 3307, 2339, 4923, 3131, 1221, 4638, 4636, 674, 1036, 4997, 2768, 7270, 6629, 3341, 8024, 4906, 3136, 1069, 1744, 5917, 4197, 2768, 7599, 3198, 8024, 791, 1921, 3300, 3119, 5966, 817, 966, 4638, 741, 872, 3766, 743, 8024, 3209, 3189, 2218, 1373, 872, 2637, 679, 2496, 1159, 8013, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

## Create Token Classifier
The value of the `language` argument determines which BERT model is used:
* Language.ENGLISH: "bert-base-uncased"
* Language.ENGLISHCASED: "bert-base-cased"
* Language.ENGLISHLARGE: "bert-large-uncased"
* Language.ENGLISHLARGECASED: "bert-large-cased"
* Language.CHINESE: "bert-base-chinese"
* Language.MULTILINGUAL: "bert-base-multilingual-cased"

Here we use the base, uncased pretrained model.

In [11]:
token_classifier = BERTTokenClassifier(language=LANGUAGE,
                                       num_labels=len(label_list),
                                       cache_dir=CACHE_DIR)

## Train Model

In [12]:
token_classifier.fit(token_ids=train_token_ids, 
                     input_mask=train_input_mask, 
                     labels=train_label_ids,
                     num_epochs=NUM_TRAIN_EPOCHS, 
                     batch_size=BATCH_SIZE, 
                     learning_rate=LEARNING_RATE)

t_total value of -1 results in schedule not being applied
Iteration:   0%|          | 0/2813 [00:00<?, ?it/s]


Iteration:  56%|█████▌    | 1574/2813 [32:13<25:26,  1.23s/it]


Epoch: 100%|██████████| 1/1 [57:34<00:00, 3454.07s/it].23s/it]

Train loss: 0.07014742273803971


## Predict on Test Data

In [13]:
predictions = token_classifier.predict(token_ids=test_token_ids, 
                                       input_mask=test_input_mask, 
                                       labels=test_label_ids, 
                                       batch_size=BATCH_SIZE,
                                       probabilities=True)

Iteration:   0%|          | 0/247 [00:00<?, ?it/s]

Iteration: 100%|██████████| 247/247 [01:40<00:00,  2.45it/s]

Evaluation loss: 0.017158769807770757


In [14]:
probabilities = predictions.probabilities
pred_label_ids = predictions.classes

## Evaluate Model
The `predict` method of the token classifier outputs label ids for all tokens, including the padded tokens. `postprocess_token_labels` is a helper function that removes the predictions on padded tokens. If a `label_map` is provided, it maps the numerical label ids back to original token labels which are usually string type. 

In [15]:
pred_tags_no_padding = postprocess_token_labels(pred_label_ids, 
                                                test_input_mask, 
                                                label_map)
true_tags_no_padding = postprocess_token_labels(test_label_ids, 
                                                test_input_mask, 
                                                label_map)
print(classification_report(true_tags_no_padding, pred_tags_no_padding, digits=2))

           precision    recall  f1-score   support

      PER       0.98      0.94      0.96      1055
      LOC       0.93      0.94      0.93      1987
      ORG       0.82      0.91      0.86       846

micro avg       0.92      0.93      0.92      3888
macro avg       0.92      0.93      0.93      3888



`postprocess_token_labels` also provides an option to remove the predictions on trailing word pieces, e.g. ##ing, so that the final predicted labels correspond to the original words in the input text. The `trailing_token_mask` is obtained from `tokenizer.preprocess_ner_tokens`

In [16]:
pred_tags_no_padding_no_trailing = postprocess_token_labels(pred_label_ids, 
                                                            test_input_mask, 
                                                            label_map, 
                                                            remove_trailing_word_pieces=True, 
                                                            trailing_token_mask=test_trailing_token_mask)
true_tags_no_padding_no_trailing = postprocess_token_labels(test_label_ids, 
                                                            test_input_mask, 
                                                            label_map, 
                                                            remove_trailing_word_pieces=True, 
                                                            trailing_token_mask=test_trailing_token_mask)
print(classification_report(true_tags_no_padding_no_trailing, pred_tags_no_padding_no_trailing, digits=2))

           precision    recall  f1-score   support

      PER       0.98      0.94      0.96      1055
      LOC       0.93      0.94      0.93      1987
      ORG       0.82      0.91      0.86       846

micro avg       0.92      0.93      0.92      3888
macro avg       0.92      0.93      0.93      3888

